# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Let's just make sure the model is loaded

#!ollama pull llama3.2:1b

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2:1b"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [6]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has a wide range of business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to create high-quality, engaging content such as articles, social media posts, product descriptions, and even entire websites. Companies like Google, Amazon, and Microsoft have already started using generative models for content generation.
2. **Automated Customer Service**: Generative AI can help automate customer service by generating responses to common customer inquiries, freeing up human agents to focus on more complex issues. This can lead to improved customer satisfaction and reduced operational costs.
3. **Personalized Marketing**: Generative AI can be used to create personalized marketing messages, offers, and content based on customers' browsing history, purchase behavior, and preferences. This can help increase conversion rates and improve customer loyalty.
4. **Design Automation**: Generative AI can generate design 

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [7]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has a wide range of business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to create high-quality content, such as articles, social media posts, and product descriptions, reducing the need for human writers and increasing efficiency.
2. **Design and Visualization**: Generative AI can generate visualizations, 3D models, and designs for products, marketing campaigns, and other creative projects, enhancing innovation and productivity.
3. **Customer Experience**: Generative AI can be used to create personalized customer experiences through chatbots, virtual assistants, and automated customer service systems.
4. **Marketing Automation**: Generative AI can help automate repetitive tasks in marketing, such as lead generation, email campaigns, and social media engagement, freeing up human marketers to focus on strategy and creativity.
5. **Predictive Maintenance**: Generative AI can analyze sensor data from m

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
from IPython.display import Markdown, display
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)


In [26]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [8]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest 
pulling aabd4debf0c8: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling c5ad996bda6e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  556 B                         
pulling 6e4c38e1172f: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions
# here used other model deepseek-r1:1.5b
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

<think>
Okay, so I'm trying to understand what underpins language models (LLMs), specifically focusing on neural networks, attention mechanisms, and transformers. I remember that LLMs are big AI systems that can generate or perform tasks like text generation, but I'm not entirely sure how they work behind the scenes. Let me start by thinking about each concept one by one.

First, let's tackle neural networks since it's a broad area. I know that most AI models, including LLMs, rely heavily on neural networks. They're often referred to as deep learning models because they have multiple layers. So, what exactly is a neural network? It sounds like a series of layers that process information through interconnected nodes or neurons. Each layer processes data and then sends the processed information (like an activation vector) to the next layer. This way, each layer can extract new features from the data it receives.

But how do these neural networks work together at different levels in LLMs?

In [ ]:
# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [22]:
deepseek_model="deepseek-r1:1.5b"

In [12]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [14]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://cloudxcellence.com")
print(ed.title)
print(ed.text)

CloudXcellence - Control your cloud
We helped AFC Ajax Amsterdam save 64% on their runtime hours.
Read customer story
Features
Features
Grip on costs
Grip on access
Grip on speed
Grip on workloads
Pricing
Pricing
Features
Features
Contact
Contact
Book a demo
Book a demo
Own your
Azure cloud,
Xcel your business.
Reduce your cloud emissions. Improve your cloud performance. Lower your cloud costs. Be alerted if something is wrong. Up and running within hours.
Making your Azure GreenOps actionable 🚀
Try it for free
Try it for free
30 day trial
Cloud emissions insights
RUNTIME HOURS
198
56%
From 455 hours last month
ERRORS
72
92%
From 907 errors last month
AZURE COSTS
€ 2.323
23%
From €2.858 last month
64%
saved on runtime hours
“CloudXcellence helped us create a complete overview of all datasources. This was needed to make a smooth switch to another provider of ticketing services. Otherwise we had to review hundreds of LogicApps manually which would have cost us a fortune!"
Read customer s

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [15]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(ed))

In [18]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [21]:
# To give you a preview -- calling OpenAI with system and user messages:

response = ollama_via_openai.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)
print(response.choices[0].message.content)

<think>
Alright, so I've got this question here: "What is 2 plus 2?" At first glance, it seems pretty straightforward. I mean, addition is a basic math concept that everyone learns early on. But wait, the user mentioned something about being snarky. Hmm, maybe they're trying to keep things light or avoid using too much technical language. 

Okay, let's break this down. The question itself doesn't seem too complicated. You just add 2 and 2 together, which equals 4. That's simple enough. I wonder if the user was thinking about more advanced math problems or something that might require a deeper understanding. Perhaps they're testing my ability to handle different types of questions beyond basic arithmetic.

But no, even at this level, I think the question is pretty direct. There are no tricks here, no substitutions, or anything like that. It's just an addition problem. Maybe the snarky part refers to something else, like being mischievous or trying to keep the conversation going. I'm not

In [23]:
response = ollama_via_openai.chat.completions.create(model=MODEL, messages=messages)
print(response.choices[0].message.content)

Wow, I'm just so excited to help with such a complex calculation. Let me put on my thinking cap and use my extensive knowledge of ancient Greek mathematics... *ahem*.

Seriously though, the answer to 2 + 2 is obviously 4. Now, if you'll excuse me, I have better things to do than hold your hand through basic arithmetic all day. Next thing you know, you'll be asking me how to breathe or something...


In [27]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [28]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled CloudXcellence - Control your cloud\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nWe helped AFC Ajax Amsterdam save 64% on their runtime hours.\nRead customer story\nFeatures\nFeatures\nGrip on costs\nGrip on access\nGrip on speed\nGrip on workloads\nPricing\nPricing\nFeatures\nFeatures\nContact\nContact\nBook a demo\nBook a demo\nOwn your\nAzure cloud,\nXcel your business.\nReduce your cloud emissions. Improve your cloud performance. Lower your cloud costs. Be alerted if something is wrong. Up and running within hours.\nMaking your Azure GreenOps actionable 🚀\nTry it for free\nTry it for free\n30 day 

In [33]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [34]:
summarize("https://cloudxcellence.com")

'### Introduction to CloudXcellence\n\nCloudXcellence is a service that helps companies control and optimize their Azure cloud infrastructure. It provides features such as cost reduction, improved performance, and enhanced security.\n\n#### Key Features\n\n* Cost optimization: reduces cloud costs by 64%\n* Performance improvement: eliminates waste by right-sizing and rebalancing workloads\n* Security improvements: boosts cloud access with real-time usage monitoring\n* Sustainability insights: tracks Azure footprint and reduces emissions\n\n#### Customer Story\n\nAFC Ajax Amsterdam, a professional club in the Netherlands, used CloudXcellence to save 64% on their runtime hours. They were facing complex operational issues due to inefficient cloud management.\n\n#### News/A announcements\n\n* CloudXcellence announced a 30-day trial period for its customers allowing them to try out the service without any commitment\n* The company also mentioned that it has supported companies in regaining 

In [35]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://cloudxcellence.com")

**CloudXcellence Overview**
Markdown summary:
CloudXcellence is a service that helps companies optimize and control their Azure cloud resources, improving efficiency, security, sustainability, and performance. It provides access to insights and features that enable companies to regain grip on their Azure cloud, reduce costs and emissions, and improve overall effectiveness.

**Key Features and Benefits**
Markdown summary:
- **Grip on Costs**: Offers cost optimization quick wins, proactive error resolution, and real-time usage monitoring.
- **Grip on Access**: Enhances security by boosting cloud access, making it efficient for operations.
- **Grip on Sustainability**: Measures Azure footprint and provides recommendations to reduce emissions.
- **Grip on Performance**: Identifies areas for right sizing and rebalancing to optimize cost-effectiveness.

**Notable News or Announcements**
Markdown summary:
- CloudXcellence helped AFC Ajax Amsterdam save 64% on their runtime hours by implementing optimized cloud costs.
- Implemented a complete overview of all datasources, making a smooth switch to another provider easier without manual error review.
- Introduced a 30-day trial for free access to the service.

**Why Choose CloudXcellence**
Markdown summary:
- Supports independent operators with flexibility in pricing plans (Basic to Enterprise), including alerts from ADF or Logic Apps.
- Offers real-time usage monitoring, helping companies optimize usage and reduce costs.
- Implements proactive error resolution for a smooth optimization journey.

In [37]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = ollama_via_openai.chat.completions.create(model=MODEL, messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! It's great to meet you. I'm excited to help and chat with you. Is there something specific you'd like to talk about or ask me? I'm all ears (or rather, all text).


In [38]:
display_summary("https://cnn.com")

The article is an update of CNN's coverage, focusing on several major news stories from June 16, 2025. Here are the key points:

1. **Middle East Conflict**: The Israeli-Palestinian conflict remains a point of tension, with airstrikes and skirmishes reported around the Gaza Strip.
2. **Arrests in France**: Four people have been arrested in connection to the murder of Vincent Vinciguia's girlfriend on social media.
3. **New Report from 688-Year-Old Murder Case**: A new investigation into the medieval case reveals a tangled web of adultery and extortion, shedding light on the lives of those involved at that time.
4. **Beirut Blast Investigation**: Research has been conducted to restore damaged artworks after a Beirut blast, highlighting efforts to preserve history and cultural heritage.

These stories are being covered in a special report for viewers around the world, offering insights into pressing global issues and local developments from June 16, 2025.

In [42]:
display_summary("https://www.invia.com")

### Invia Group Overview

*   The website provides an overview of the company, emphasizing its focus on providing holiday expert services across Europe and promoting a large network.
    *   **Company Structure**: Invia is described as "THE" provider for holidays in the industry. It also mentions starting a growing business.

### Careers at Invia

*   The section focuses on exploring available careers within the company, highlighting growth opportunities.
    *   **Job Description**: There's no explicit job description provided, but it can be inferred that career openings might involve working closely with customers and other teams to contribute to the company's mission.

### Upcoming Year: Invia Group 2025

*   This section is likely related to the company's plan for the upcoming year.
    *   **Key Highlights**: As no specific details are mentioned, some assumptions have been made:

        :   The "Invia Group 2025" refers to an outline of the company's ambitions for future growth and development.
            :   These might include expanding its customer base, enhancing services, or exploring international collaborations.

   